In [2]:
import datetime as dt

import eranest

In [3]:
# eranest.era5ify_geojson(request_id, variables, start_date, end_date, json_file, frequency, resolution)
# request_id : str, unique identifier for the request
# variables : list, list of variables to download
# start_date : datetime, start date of the data
# end_date : datetime, end date of the data
# json_file : str, path to the geojson file containing the area of interest
# frequency : str, frequency of the data (hourly, daily, weekly, monthly, yearly), optional (hourly by default)
# resolution : float, resolution of the data in degrees (0.1, 0.25, etc.), optional (0.25 by default)

In [4]:
df = eranest.era5ify_geojson(
    request_id="test",
    variables=[
        "2m_temperature",
        "total_precipitation",
        "surface_pressure",
        "2m_dewpoint_temperature",
    ],
    start_date=dt.datetime(2023, 1, 1),
    end_date=dt.datetime(2023, 1, 31),
    json_file="../data/india.json",
    frequency="daily",
    resolution="0.25",
)

Successfully loaded JSON file with utf-8 encoding
Valid GeoJSON detected: ../data/india.json
✓ CDS API configuration is already set up and valid.

STARTING ERA5 DATA PROCESSING
Request ID: test
Variables: ['2m_temperature', 'total_precipitation', 'surface_pressure', '2m_dewpoint_temperature']
Date Range: 2023-01-01 to 2023-01-31
Frequency: daily
Resolution: 0.25°
GeoJSON File: ../data/india.json

--- Input Validation ---
✓ All inputs validated successfully

--- Loading GeoJSON File ---
Attempting to load: ../data/india.json
  Trying encoding 1/4: utf-8
✓ Successfully loaded GeoJSON file with utf-8 encoding
✓ GeoJSON contains 1 feature(s)

--- Calculating Bounding Box ---
✓ Bounding Box calculated:
  North: 35.4940°
  South: 7.9655°
  East:  97.4026°
  West:  68.1766°
  Area:  29.2259° × 27.5285°

--- Determining Processing Strategy ---
Using monthly dataset: False
Total days to process: 31
Max days per chunk: 14
Needs chunking: True
Will process 3 chunks C2

CHUNK 1/3
Processing: 2023-

2025-05-29 22:33:50,800 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-29 22:33:51,721 INFO Request ID is 4fb482aa-ef8a-4cc3-8f46-76633743138f
2025-05-29 22:33:51,926 INFO status has been updated to accepted
2025-05-29 22:34:13,945 INFO status has been updated to running
2025-05-29 22:34:42,921 INFO status has been updated to successful


Download complete: test_chunk1.zip
  ✓ Download completed: test_chunk1.zip
  → Extracting files...
Extracting zip file: test_chunk1.zip
Extracted NetCDF files:
  - d:\eranest\notebooks\test_chunk1\data_stream-oper_stepType-accum.nc
  - d:\eranest\notebooks\test_chunk1\data_stream-oper_stepType-instant.nc
  ✓ Extracted 2 files
  → Processing NetCDF files...
    Processing file 1/2: data_stream-oper_stepType-accum.nc


D:\eranest\eranest\core.py:1590: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"    ✓ Loaded dataset with shape: {dict(ds.dims)}")
D:\eranest\eranest\core.py:1590: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"    ✓ Loaded dataset with shape: {dict(ds.dims)}")
D:\eranest\eranest\core.py:1606: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"  ✓ Merged dataset shape: {dict

    ✓ Loaded dataset with shape: {'valid_time': 336, 'latitude': 111, 'longitude': 117}
    Processing file 2/2: data_stream-oper_stepType-instant.nc
    ✓ Loaded dataset with shape: {'valid_time': 336, 'latitude': 111, 'longitude': 117}
  → Merging datasets...
  ✓ Merged dataset shape: {'valid_time': 336, 'latitude': 111, 'longitude': 117}
  → Filtering by shapefile...
Starting optimized filtering process...
→ Extracting unique lat/lon coordinates from dataset...
✓ Found 12987 unique lat/lon combinations
→ Filtering unique coordinates against polygon...
✓ Coordinate filtering completed in 0.51 seconds
  - Points inside: 4446
  - Points outside: 8541
  - Percentage inside: 34.23%
→ Filtering original dataset using inside coordinates...
  Converting dataset to DataFrame...
  ✓ Converted to DataFrame with 4363632 rows
  ✓ Created lookup set with 4446 coordinate pairs
  Filtering DataFrame rows...
  ✓ Filtered from 4363632 to 1493856 rows
✓ Dataset filtering completed in 2.96 seconds

---

2025-05-29 22:35:35,249 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-29 22:35:36,172 INFO Request ID is 4ea21775-fdc0-4372-a73f-4535dd60f736
2025-05-29 22:35:36,377 INFO status has been updated to accepted
2025-05-29 22:36:53,176 INFO status has been updated to running
2025-05-29 22:37:31,810 INFO status has been updated to successful
D:\eranest\eranest\core.py:1590: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"    ✓ Loaded dataset with shape: {dict(ds.dims)}")
D:\eranest\eranest\core.py:1590: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension name

Download complete: test_chunk2.zip
  ✓ Download completed: test_chunk2.zip
  → Extracting files...
Extracting zip file: test_chunk2.zip
Extracted NetCDF files:
  - d:\eranest\notebooks\test_chunk2\data_stream-oper_stepType-accum.nc
  - d:\eranest\notebooks\test_chunk2\data_stream-oper_stepType-instant.nc
  ✓ Extracted 2 files
  → Processing NetCDF files...
    Processing file 1/2: data_stream-oper_stepType-accum.nc
    ✓ Loaded dataset with shape: {'valid_time': 336, 'latitude': 111, 'longitude': 117}
    Processing file 2/2: data_stream-oper_stepType-instant.nc
    ✓ Loaded dataset with shape: {'valid_time': 336, 'latitude': 111, 'longitude': 117}
  → Merging datasets...
  ✓ Merged dataset shape: {'valid_time': 336, 'latitude': 111, 'longitude': 117}
  → Filtering by shapefile...
Starting optimized filtering process...
→ Extracting unique lat/lon coordinates from dataset...
✓ Found 12987 unique lat/lon combinations
→ Filtering unique coordinates against polygon...
✓ Coordinate filteri

2025-05-29 22:41:20,650 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-29 22:41:22,085 INFO Request ID is c7d10e2f-8107-4ea6-b2ea-19fb673faac6
2025-05-29 22:41:22,286 INFO status has been updated to accepted
2025-05-29 22:42:13,181 INFO status has been updated to running
2025-05-29 22:42:38,993 INFO status has been updated to successful
D:\eranest\eranest\core.py:1590: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"    ✓ Loaded dataset with shape: {dict(ds.dims)}")
D:\eranest\eranest\core.py:1590: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension name

Download complete: test_chunk3.zip
  ✓ Download completed: test_chunk3.zip
  → Extracting files...
Extracting zip file: test_chunk3.zip
Extracted NetCDF files:
  - d:\eranest\notebooks\test_chunk3\data_stream-oper_stepType-accum.nc
  - d:\eranest\notebooks\test_chunk3\data_stream-oper_stepType-instant.nc
  ✓ Extracted 2 files
  → Processing NetCDF files...
    Processing file 1/2: data_stream-oper_stepType-accum.nc
    ✓ Loaded dataset with shape: {'valid_time': 72, 'latitude': 111, 'longitude': 117}
    Processing file 2/2: data_stream-oper_stepType-instant.nc
    ✓ Loaded dataset with shape: {'valid_time': 72, 'latitude': 111, 'longitude': 117}
  → Merging datasets...
  ✓ Merged dataset shape: {'valid_time': 72, 'latitude': 111, 'longitude': 117}
  → Filtering by shapefile...
Starting optimized filtering process...
→ Extracting unique lat/lon coordinates from dataset...
✓ Found 12987 unique lat/lon combinations
→ Filtering unique coordinates against polygon...
✓ Coordinate filtering 

In [5]:
df = eranest.era5ify_geojson(
    request_id="test2",
    variables=[
        "2m_temperature", 
        "total_precipitation"
    ],
    start_date=dt.datetime(2024, 1, 1),
    end_date=dt.datetime(2024, 12, 31),
    json_file="../data/latvia.geojson",
    frequency="monthly",
    resolution="0.1",
)

Successfully loaded JSON file with utf-8 encoding
Valid GeoJSON detected: ../data/latvia.geojson
✓ CDS API configuration is already set up and valid.

STARTING ERA5 DATA PROCESSING
Request ID: test2
Variables: ['2m_temperature', 'total_precipitation']
Date Range: 2024-01-01 to 2024-12-31
Frequency: monthly
Resolution: 0.1°
GeoJSON File: ../data/latvia.geojson

--- Input Validation ---
✓ All inputs validated successfully

--- Loading GeoJSON File ---
Attempting to load: ../data/latvia.geojson
  Trying encoding 1/4: utf-8
✓ Successfully loaded GeoJSON file with utf-8 encoding
✓ GeoJSON contains 1 feature(s)

--- Calculating Bounding Box ---
✓ Bounding Box calculated:
  North: 58.0856°
  South: 55.6776°
  East:  28.2431°
  West:  20.9537°
  Area:  7.2894° × 2.4080°

--- Determining Processing Strategy ---
Using monthly dataset: True
Total months to process: 12
Max months per chunk: 10
Needs chunking: True
Will process 2 chunks C1

CHUNK 1/2
Processing: 2024-01-01 to 2024-10-31
  → Downloa

2025-05-29 22:50:30,167 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-29 22:50:30,853 INFO Request ID is 9004e213-bd94-4f44-b480-1126f2b0fece
2025-05-29 22:50:31,057 INFO status has been updated to accepted
2025-05-29 22:50:45,702 INFO status has been updated to running
2025-05-29 22:50:53,791 INFO status has been updated to successful
D:\eranest\eranest\core.py:1478: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"    ✓ Loaded dataset with shape: {dict(ds.dims)}")
D:\eranest\eranest\core.py:1478: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension name

Download complete: test2_chunk1.zip
  ✓ Download completed: test2_chunk1.zip
  → Extracting files...
Extracting zip file: test2_chunk1.zip
Extracted NetCDF files:
  - d:\eranest\notebooks\test2_chunk1\data_0.nc
  - d:\eranest\notebooks\test2_chunk1\data_1.nc
  ✓ Extracted 2 files
  → Processing NetCDF files...
    Processing file 1/2: data_0.nc
    ✓ Loaded dataset with shape: {'valid_time': 10, 'latitude': 25, 'longitude': 73}
    Processing file 2/2: data_1.nc
    ✓ Loaded dataset with shape: {'valid_time': 10, 'latitude': 25, 'longitude': 73}
  → Merging datasets...
  ✓ Merged dataset shape: {'valid_time': 20, 'latitude': 25, 'longitude': 73}
  → Filtering by shapefile...
Starting optimized filtering process...
→ Extracting unique lat/lon coordinates from dataset...
✓ Found 1825 unique lat/lon combinations
→ Filtering unique coordinates against polygon...
✓ Coordinate filtering completed in 0.21 seconds
  - Points inside: 962
  - Points outside: 863
  - Percentage inside: 52.71%
→ F

2025-05-29 22:51:04,441 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-29 22:51:05,071 INFO Request ID is 1975f8ac-b71d-4009-b5b3-45c93fa415c7
2025-05-29 22:51:05,262 INFO status has been updated to accepted
2025-05-29 22:51:27,482 INFO status has been updated to running
2025-05-29 22:51:39,053 INFO status has been updated to successful
D:\eranest\eranest\core.py:1478: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"    ✓ Loaded dataset with shape: {dict(ds.dims)}")
D:\eranest\eranest\core.py:1478: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension name

Download complete: test2_chunk2.zip
  ✓ Download completed: test2_chunk2.zip
  → Extracting files...
Extracting zip file: test2_chunk2.zip
Extracted NetCDF files:
  - d:\eranest\notebooks\test2_chunk2\data_0.nc
  - d:\eranest\notebooks\test2_chunk2\data_1.nc
  ✓ Extracted 2 files
  → Processing NetCDF files...
    Processing file 1/2: data_0.nc
    ✓ Loaded dataset with shape: {'valid_time': 2, 'latitude': 25, 'longitude': 73}
    Processing file 2/2: data_1.nc
    ✓ Loaded dataset with shape: {'valid_time': 2, 'latitude': 25, 'longitude': 73}
  → Merging datasets...
  ✓ Merged dataset shape: {'valid_time': 4, 'latitude': 25, 'longitude': 73}
  → Filtering by shapefile...
Starting optimized filtering process...
→ Extracting unique lat/lon coordinates from dataset...
✓ Found 1825 unique lat/lon combinations
→ Filtering unique coordinates against polygon...
✓ Coordinate filtering completed in 0.21 seconds
  - Points inside: 962
  - Points outside: 863
  - Percentage inside: 52.71%
→ Filt

In [6]:
# eranest.era5ify_bbox(request_id, variables, start_date, end_date, north, south, east, west, frequency, resolution)
# request_id : str, unique identifier for the request
# variables : list, list of variables to download
# start_date : datetime, start date of the data
# end_date : datetime, end date of the data
# north : float, northern boundary of the bounding box
# south : float, southern boundary of the bounding box
# east : float, eastern boundary of the bounding box
# west : float, western boundary of the bounding box
# frequency : str, frequency of the data (hourly, daily, weekly, monthly, yearly), optional (hourly by default)
# resolution : float, resolution of the data in degrees (0.1, 0.25, etc.), optional (0.25 by default)

In [7]:
df = eranest.era5ify_bbox(
    request_id="test3",
    variables=[
        "2m_temperature", 
        "total_precipitation"
    ],
    start_date=dt.datetime(2024, 1, 1),
    end_date=dt.datetime(2024, 1, 15),
    north = 30,
    south = 20,
    east = 80,
    west = 70,
    frequency="daily",
    resolution=0.25
)


STARTING ERA5 BBOX PROCESSING
Request ID: test3
Variables: ['2m_temperature', 'total_precipitation']
Date Range: 2024-01-01 to 2024-01-15
Bounding Box: N:30, S:20, E:80, W:70
Frequency: daily
Resolution: 0.25°

--- Bounding Box Validation ---
✓ Bounding box coordinates validated successfully
  Area: 10.0000° × 10.0000°

--- Creating GeoJSON from Bounding Box ---
✓ GeoJSON created successfully
✓ Created temporary GeoJSON file: C:\Users\ATHARV~1\AppData\Local\Temp\test3_temp_geojson.json

--- Delegating to Main Processing Function ---
✓ CDS API configuration is already set up and valid.

STARTING ERA5 DATA PROCESSING
Request ID: test3
Variables: ['2m_temperature', 'total_precipitation']
Date Range: 2024-01-01 to 2024-01-15
Frequency: daily
Resolution: 0.25°
GeoJSON File: C:\Users\ATHARV~1\AppData\Local\Temp\test3_temp_geojson.json

--- Input Validation ---
✓ All inputs validated successfully

--- Loading GeoJSON File ---
Attempting to load: C:\Users\ATHARV~1\AppData\Local\Temp\test3_tem

2025-05-29 22:52:26,337 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-29 22:52:27,183 INFO Request ID is 872c4878-2d0d-4982-9a16-de2e1d5822ae
2025-05-29 22:52:27,410 INFO status has been updated to accepted
2025-05-29 22:52:41,723 INFO status has been updated to running
2025-05-29 22:53:18,384 INFO status has been updated to successful
D:\eranest\eranest\core.py:1590: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"    ✓ Loaded dataset with shape: {dict(ds.dims)}")
D:\eranest\eranest\core.py:1590: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension name

Download complete: test3_chunk1.zip
  ✓ Download completed: test3_chunk1.zip
  → Extracting files...
Extracting zip file: test3_chunk1.zip
Extracted NetCDF files:
  - d:\eranest\notebooks\test3_chunk1\data_stream-oper_stepType-accum.nc
  - d:\eranest\notebooks\test3_chunk1\data_stream-oper_stepType-instant.nc
  ✓ Extracted 2 files
  → Processing NetCDF files...
    Processing file 1/2: data_stream-oper_stepType-accum.nc
    ✓ Loaded dataset with shape: {'valid_time': 336, 'latitude': 41, 'longitude': 41}
    Processing file 2/2: data_stream-oper_stepType-instant.nc
    ✓ Loaded dataset with shape: {'valid_time': 336, 'latitude': 41, 'longitude': 41}
  → Merging datasets...
  ✓ Merged dataset shape: {'valid_time': 336, 'latitude': 41, 'longitude': 41}
  → Filtering by shapefile...
Starting optimized filtering process...
→ Extracting unique lat/lon coordinates from dataset...
✓ Found 1681 unique lat/lon combinations
→ Filtering unique coordinates against polygon...
✓ Coordinate filtering

2025-05-29 22:53:32,105 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-29 22:53:33,090 INFO Request ID is 42ccc1b7-9318-4e59-a9f8-7fe54242ad33
2025-05-29 22:53:33,253 INFO status has been updated to accepted
2025-05-29 22:53:55,452 INFO status has been updated to running
2025-05-29 22:54:07,128 INFO status has been updated to successful
D:\eranest\eranest\core.py:1590: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"    ✓ Loaded dataset with shape: {dict(ds.dims)}")
D:\eranest\eranest\core.py:1590: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension name

Download complete: test3_chunk2.zip
  ✓ Download completed: test3_chunk2.zip
  → Extracting files...
Extracting zip file: test3_chunk2.zip
Extracted NetCDF files:
  - d:\eranest\notebooks\test3_chunk2\data_stream-oper_stepType-accum.nc
  - d:\eranest\notebooks\test3_chunk2\data_stream-oper_stepType-instant.nc
  ✓ Extracted 2 files
  → Processing NetCDF files...
    Processing file 1/2: data_stream-oper_stepType-accum.nc
    ✓ Loaded dataset with shape: {'valid_time': 24, 'latitude': 41, 'longitude': 41}
    Processing file 2/2: data_stream-oper_stepType-instant.nc
    ✓ Loaded dataset with shape: {'valid_time': 24, 'latitude': 41, 'longitude': 41}
  → Merging datasets...
  ✓ Merged dataset shape: {'valid_time': 24, 'latitude': 41, 'longitude': 41}
  → Filtering by shapefile...
Starting optimized filtering process...
→ Extracting unique lat/lon coordinates from dataset...
✓ Found 1681 unique lat/lon combinations
→ Filtering unique coordinates against polygon...
✓ Coordinate filtering co